In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import re
pd.set_option('display.max_seq_items', None)

Load Data

In [ ]:
admissions = pd.read_csv('./admissions.csv')
admissions = admissions[['hadm_id','deathtime','hospital_expire_flag']]
admissions['deathtime'] = admissions['deathtime'].fillna(0)
dead_index = admissions[(admissions['deathtime']!=0) & (admissions['hospital_expire_flag']==1)].index
admissions['Death'] = 0
admissions['Death'][dead_index] = 1

death = admissions[['hadm_id','Death']]

patients = pd.read_csv('./patients.csv')
icustays = pd.read_csv('./icustays.csv')
patients = patients[['subject_id','gender','anchor_age']]

vital_lab = pd.read_csv('./10min vital lab Dataset.csv')
Ketamine = pd.read_csv('./10min Ketamine.csv')
Propofol = pd.read_csv('./10min Propofol.csv')
Midazolam = pd.read_csv('./10min Midazolam.csv')
Fentanyl = pd.read_csv('./10min Fentanyl.csv')
fluids = pd.read_csv('./10min Fluids.csv')
urine = pd.read_csv('./10min Urine output.csv')
LR_NS_Pl = pd.read_csv('./10min LR_NS_TR_AL.csv')
vasopressors = pd.read_csv('./10min Vasopressors.csv')
transfusion = pd.read_csv('./10min transfusion.csv')
transfusion.rename(columns={'tranfusion':'Transfusion'}, inplace=True)
Epinephrine = pd.read_csv('./10min Epinephrine.csv')
Norepinephrine = pd.read_csv('./10min Norepinephrine.csv')
Phenylephrine = pd.read_csv('./10min Phenylephrine.csv')
Vasopressin = pd.read_csv('./10min Vasopressin.csv')

In [ ]:
print(vital_lab['stay_id'].unique().shape)
print(Ketamine['stay_id'].unique().shape)
print(Propofol['stay_id'].unique().shape)
print(Midazolam['stay_id'].unique().shape)
print(Fentanyl['stay_id'].unique().shape)
print(fluids['stay_id'].unique().shape)
print(urine['stay_id'].unique().shape)
print(LR_NS_Pl['stay_id'].unique().shape)
print(vasopressors['stay_id'].unique().shape)
print(transfusion['stay_id'].unique().shape)
print(Epinephrine['stay_id'].unique().shape)
print(Norepinephrine['stay_id'].unique().shape)
print(Phenylephrine['stay_id'].unique().shape)
print(Vasopressin['stay_id'].unique().shape)

In [ ]:
icustays

Merging All

In [ ]:
main = pd.merge(vital_lab, icustays, on=['subject_id', 'hadm_id','stay_id'], how='left')
main = pd.merge(main, LR_NS_Pl, on=['subject_id', 'hadm_id','stay_id', 'time'], how='left')
main = pd.merge(main, vasopressors, on=['subject_id', 'hadm_id','stay_id', 'time'], how='left')

main = pd.merge(main, Epinephrine, on=['subject_id', 'hadm_id','stay_id', 'time'], how='left')
main = pd.merge(main, Norepinephrine, on=['subject_id', 'hadm_id','stay_id', 'time'], how='left')
main = pd.merge(main, Phenylephrine, on=['subject_id', 'hadm_id','stay_id', 'time'], how='left')
main = pd.merge(main, Vasopressin, on=['subject_id', 'hadm_id','stay_id', 'time'], how='left')
main = pd.merge(main, transfusion, on=['subject_id', 'hadm_id','stay_id', 'time'], how='left')
main = pd.merge(main, Ketamine, on=['subject_id', 'hadm_id','stay_id', 'time'], how='left')
main = pd.merge(main, Propofol, on=['subject_id', 'hadm_id','stay_id', 'time'], how='left')
main = pd.merge(main, Midazolam, on=['subject_id', 'hadm_id','stay_id', 'time'], how='left')
main = pd.merge(main, Fentanyl, on=['subject_id', 'hadm_id','stay_id', 'time'], how='left')
main = pd.merge(main, fluids, on=['subject_id', 'hadm_id','stay_id', 'time'], how='left')
main = pd.merge(main, urine, on=['subject_id', 'hadm_id','stay_id', 'time'], how='left')


main = pd.merge(main, patients, on=['subject_id'], how='left')

main = pd.merge(main, death, on=['hadm_id'], how='left')

#main = main[main['LOS']>1].reset_index(drop=True)

main['stay_id'].unique().shape

In [ ]:
main.columns

In [ ]:
def fillna_height(x):
    x.reset_index(drop=True, inplace=True)
    height_mean = x['Height'].mean()

    x['Height'] = height_mean

    return x

main = main.groupby('stay_id').apply(fillna_height)
main.reset_index(drop=True, inplace=True)
'''
male_index = main[main['gender']=='M'].index
female_index = main[main['gender']=='F'].index

main['Ideal Weight'] = 0
main['Ideal Weight'][male_index] = 50 + 2.3*(main['Height'][male_index]/2.54)/5
main['Ideal Weight'][female_index] = 45.5 + 2.3*(main['Height'][female_index]/2.54)/5

main = main.sort_values(by=['subject_id','hadm_id','stay_id', 'time'], ascending=[True,True,True,True])
main.reset_index(drop=True, inplace=True)
'''

Imputation: Forward fill

In [ ]:
main['Actual Troponin test'] = 0 
actual_trop_index = main[main['Troponin'].notna()].index
main['Actual Troponin test'][actual_trop_index] = 1

main['Actual Creatinine test'] = 0 
actual_creatinine_index = main[main['Creatinine'].notna()].index
main['Actual Creatinine test'][actual_creatinine_index] = 1

main['Actual Lactate test'] = 0
actual_Lactate_index = main[main['Lactate'].notna()].index
main['Actual Lactate test'][actual_Lactate_index] = 1

In [ ]:
main['IBlood pressure systolic'] = main['IBlood pressure systolic'].fillna(-1)
main['IBlood pressure diastolic'] = main['IBlood pressure diastolic'].fillna(-1)
main['IV fluids'] = main['IV fluids'].fillna(0)
main['Vasopressors'] = main['Vasopressors'].fillna(0)
main['Transfusion'] = main['Transfusion'].fillna(0)

main['Epinephrine'] = main['Epinephrine'].fillna(0)
main['Norepinephrine'] = main['Norepinephrine'].fillna(0)
main['Phenylephrine'] = main['Phenylephrine'].fillna(0)
main['Vasopressin'] = main['Vasopressin'].fillna(0)
main['Ketamine'] = main['Ketamine'].fillna(0)
main['Propofol'] = main['Propofol'].fillna(0)
main['Midazolam'] = main['Midazolam'].fillna(0)
main['Fentanyl'] = main['Fentanyl'].fillna(0)
main['Urine Output'] = main['Urine Output'].fillna(0)
main['LR_NS'] = main['LR_NS'].fillna(0)


main = main[['subject_id', 'hadm_id','stay_id', 'time', 'intime', 'outtime',
             'Death', 'gender', 'anchor_age', 'Weight', 'Height', 
             'Heart rate', 'NIBlood pressure systolic', 'NIBlood pressure diastolic', 'IBlood pressure systolic', 'IBlood pressure diastolic', 'Respiratory rate', 'SpO2', 'Temperature', 
             'Hemoglobin', 'Creatinine', 'Troponin', 'Lactate', 
             'Ketamine', 'Propofol', 'Midazolam', 'Fentanyl', 
             'IV fluids', 'Urine Output',
             'LR_NS', 'Transfusion', 'Vasopressors', 'Epinephrine', 'Norepinephrine', 'Phenylephrine', 'Vasopressin',
             'Actual Troponin test', 'Actual Creatinine test', 'Actual Lactate test'
             ]]

In [ ]:
# 간격 설정 보간
def imputation(x):
    x.reset_index(drop=True, inplace=True)

    #interval_backward = 6*6

    x = x.fillna(method='ffill')

    #x.loc[0:interval_backward] = x.loc[0:interval_backward].fillna(method='bfill')

    return x

imputated_data = main.groupby('stay_id').apply(imputation)
main = 0
imputated_data.reset_index(drop=True, inplace=True)
print(imputated_data['stay_id'].unique().shape)

In [ ]:
imputated_data.shape

Exclude Nan data in crucial feature(for labeling)

In [ ]:
def Bye_bye(x): 
    x.reset_index(drop=True, inplace=True)
    column_list = ['NIBlood pressure systolic', 'NIBlood pressure diastolic']
    
    has_nan = x[column_list].isna().any(axis=1).any()

    if has_nan == False:
        return x
    
    else :
        df_notna = x.dropna(subset=column_list)
        df_notna.reset_index(drop=True, inplace=True)

        df_isna = x[x[column_list].isnull().any(axis=1)]

        if len(df_notna)!=0:
            if df_notna['Actual Troponin test'][0]==0:
                df_notna['Actual Troponin test'][0] = df_isna['Actual Troponin test'].max()

            if df_notna['Actual Creatinine test'][0]==0: 
                df_notna['Actual Creatinine test'][0] = df_isna['Actual Creatinine test'].max()

            if df_notna['Actual Lactate test'][0]==0:
                df_notna['Actual Lactate test'][0] = df_isna['Actual Lactate test'].max()

            return df_notna

cleansed_data = imputated_data.groupby('stay_id').apply(Bye_bye)
print(cleansed_data.shape)
#imputated_data = 0
cleansed_data.reset_index(drop=True, inplace=True)
print(cleansed_data['stay_id'].unique().shape)

cleansed_data.to_csv('./10min cleansed Dataset.csv', index=False)

In [ ]:
cleansed_data.isnull().sum()

In [ ]:
%reset -f

Definitive features

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import re
pd.set_option('display.max_seq_items', None)

In [ ]:
data = pd.read_csv('./10min cleansed Dataset.csv')

print(data['stay_id'].unique().shape)
nan_columns = data.columns[data.isnull().any()].tolist()
print("Columns with NaN values :", nan_columns)
data.isnull().sum().sum()

In [ ]:
data.isnull().sum()

In [ ]:
#Non-Invasive and Invasive blood pressure
#Check Flash value
flash_condition1 = (data['IBlood pressure systolic']*0.9 <= data['IBlood pressure diastolic'])
flash_condition2 = (data['IBlood pressure systolic']!=-1)
flash_condition3 = (data['IBlood pressure diastolic']!=-1)
flash_index = data[flash_condition1 & flash_condition2 & flash_condition3].index
data['IBlood pressure systolic'][flash_index] = -1
data['IBlood pressure diastolic'][flash_index] = -1

#handle invasive and noninvasive
invasive_sbp_index = data[data['IBlood pressure systolic']!=-1].index
invasive_dbp_index = data[data['IBlood pressure diastolic']!=-1].index

data['SBP type'] = 'NI'
data['DBP type'] = 'NI'
data['NIBlood pressure systolic'][invasive_sbp_index] = data['IBlood pressure systolic'][invasive_sbp_index]
data['NIBlood pressure diastolic'][invasive_dbp_index] = data['IBlood pressure diastolic'][invasive_dbp_index]
data['SBP type'][invasive_sbp_index] = 'I'
data['DBP type'][invasive_dbp_index] = 'I'

data.drop(columns=['IBlood pressure systolic', 'IBlood pressure diastolic'], inplace=True)
data.rename(columns={'NIBlood pressure systolic':'Blood pressure systolic'}, inplace=True)
data.rename(columns={'NIBlood pressure diastolic':'Blood pressure diastolic'}, inplace=True)

In [ ]:
#MBP
data['Blood pressure mean'] = data['Blood pressure systolic']+(data['Blood pressure diastolic']-data['Blood pressure systolic'])/3

data.to_csv('./10min Dataset for shock labeling.csv', index=False)

In [ ]:
%reset -f

Define Shock

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import re
import math

In [ ]:
data = pd.read_csv('./Data preprocessed 4/10min Dataset for shock labeling.csv')

In [ ]:
data.columns

In [ ]:
data.isnull().sum()

In [ ]:
def labeling_peri_shock_weak(x):
    x.reset_index(drop=True, inplace=True)
    x['peri_Shock'] = 0

    for p in range(len(x)):
        if ((x['Blood pressure mean'][p]>65) & (x['Blood pressure mean'][p]<75)):               
            x['peri_Shock'][p] = 1

    return x

def labeling_shock_weak(x):
    x.reset_index(drop=True, inplace=True)
    x['Shock'] = 0
    x['Ambiguous Shock'] = 0

    for p in range(len(x)):
        if not math.isnan(x['Lactate'][p]):
            if ((x['Blood pressure mean'][p]<=65) & (x['Lactate'][p]>=2)) | (x['Vasopressors'][p] > 0):               
                x['Shock'][p] = 1
        else:
            if x['Vasopressors'][p] > 0 :
                x['Shock'][p] = 1
            elif x['Blood pressure mean'][p]<=65:               
                x['Ambiguous Shock'][p] = 1 

    return x

def num_shock(x):
    x.reset_index(drop=True, inplace=True)
    new_df = pd.DataFrame(index=[[0]], columns=['stay_id','Shock','Death','Shock and Death','Ambiguous Shock'])

    new_df['stay_id'][0] = x['stay_id'].max()
    new_df['Shock'][0] = x['Shock'].max()
    new_df['Ambiguous Shock'][0] = x['Shock'].max()
    new_df['Death'][0] = x['Death'].max()
    if (new_df['Shock'][0]==1) & (new_df['Death'][0]==1) :
        new_df['Shock and Death'][0] = 1
    else :
        new_df['Shock and Death'][0] = 0

    return new_df

In [ ]:
result0 = data.groupby('stay_id').apply(labeling_peri_shock_weak)
result0.reset_index(drop=True, inplace=True)
print(result0['stay_id'].unique().shape)
print(result0.shape)

result1 = result0.groupby('stay_id').apply(labeling_shock_weak)
result1.reset_index(drop=True, inplace=True)
print(result1['stay_id'].unique().shape)
print(result1.shape)

result2 = result1.groupby('stay_id').apply(num_shock)
shock_sum = result2['Shock'].sum()
ambiguous_shock_sum = result2['Ambiguous Shock'].sum()
shock_death_sum = result2['Shock and Death'].sum()

print(result0['stay_id'].unique().shape)
print(result1['stay_id'].unique().shape)
print(shock_sum)
print(ambiguous_shock_sum)
print(shock_death_sum)

In [ ]:
result1.to_csv('./10min Shock Dataset labeling.csv', index=False)

In [ ]:
print(result1['peri_Shock'].unique())
print(result1['Shock'].unique())
print(result1['Ambiguous Shock'].unique())

In [ ]:
%reset -f

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import re
import math

In [ ]:
result1 = pd.read_csv('./10min Shock Dataset labeling.csv')

In [ ]:
def shock_patient(x):
    x.reset_index(drop=True, inplace=True)
    if x['Shock'].max() == 1:
        return x
    
result3 = result1.groupby('stay_id').apply(shock_patient)
result3.reset_index(drop=True, inplace=True)
print(result3['stay_id'].unique().shape)
print(result3.shape)

In [ ]:
result3['time'] = pd.to_datetime(result3['time'])
result3['intime'] = pd.to_datetime(result3['intime'])
result3['outtime'] = pd.to_datetime(result3['outtime'])

result3['Shock index'] = result3['Heart rate']/result3['Blood pressure systolic']

In [ ]:
def resampling_func_1hr(x):
    mean_name = ['Weight', 'Height', 'Heart rate', 'Blood pressure systolic', 'Blood pressure diastolic', 'Blood pressure mean', 'Respiratory rate', 'SpO2', 'Temperature', 'Shock index',
                    'Hemoglobin', 'Creatinine', 'Troponin', 'Lactate']
    sum_name = ['Propofol', 'Midazolam', 'Fentanyl', 'IV fluids', 'Urine Output', 'LR_NS', 'Transfusion', 'Shock', 'Ambiguous Shock', 'peri_Shock']
    max_name = ['Ketamine','Vasopressors','Epinephrine','Norepinephrine','Phenylephrine','Vasopressin', 'Actual Troponin test', 'Actual Creatinine test', 'Actual Lactate test']

    column = ['subject_id','hadm_id','stay_id', 'time', 'Death', 'gender', 'anchor_age'] + mean_name + sum_name + max_name

    x.reset_index(drop=True, inplace=True)
    date_range = pd.date_range(x['intime'][0], x['outtime'][0], freq='1H', inclusive='right')
    new_df = pd.DataFrame(index=date_range, columns=column)
    
    new_df['subject_id'] = x['subject_id'][0]
    new_df['hadm_id'] = x['hadm_id'][0]
    new_df['stay_id'] = x['stay_id'][0]
    new_df['time'] = new_df.index
    new_df['Death'] = x['Death'][0]
    new_df['gender'] = x['gender'][0]
    new_df['anchor_age'] = x['anchor_age'][0]
    
    for i in range(len(date_range)):
        if i == 0:
            condition1 = date_range[i]>=x['time']
            sa = x[condition1]
            for p in mean_name:
                new_df[p][i] = sa[p].mean()
        else :
            condition2 = date_range[i-1]<x['time']
            condition3 = date_range[i]>=x['time']
            sa = x[(condition2)&(condition3)]

            for p in mean_name:
                new_df[p][i] = sa[p].mean()

    for i in range(len(date_range)):
        if i == 0:
            condition1 = date_range[i]>=x['time']
            sa = x[condition1]
            for p in sum_name:
                new_df[p][i] = sa[p].sum()
        else :
            condition2 = date_range[i-1]<x['time']
            condition3 = date_range[i]>=x['time']
            sa = x[(condition2)&(condition3)]

            for p in sum_name:
                new_df[p][i] = sa[p].sum()

    for i in range(len(date_range)):
        if i == 0:
            condition1 = date_range[i]>=x['time']
            sa = x[condition1]
            for p in max_name:
                new_df[p][i] = sa[p].max()
        else :
            condition2 = date_range[i-1]<x['time']
            condition3 = date_range[i]>=x['time']
            sa = x[(condition2)&(condition3)]

            for p in max_name:
                new_df[p][i] = sa[p].max()
    
    return new_df

result_1hr = result3.groupby('stay_id').apply(resampling_func_1hr)
result_1hr = result_1hr.reset_index(drop=True)
print('---------------------------------------------------------------------------------')

In [ ]:
result_1hr.to_csv('./1hr Shock Dataset resampled labeled.csv', index=False)

In [ ]:
def add_elevation_index(x):
    x.reset_index(drop=True, inplace=True)
    x['Troponin Elevation index'] = None
    x['Creatinine Elevation index'] = None
    x['Lactate Elevation index'] = None

    #troponin
    actual_trop_test_index = x[x['Actual Troponin test']==1].index
    atti_ls = actual_trop_test_index.tolist()
    #index를 list로 변환해서 길이만큼
    if len(atti_ls)!=0:
        x['Troponin Elevation index'][atti_ls[0]:] = 'maintain'
    for i in range(len(atti_ls)-1):
        before = x['Troponin'][atti_ls[i]]
        after = x['Troponin'][atti_ls[i+1]]
        if after > before*1.2:
            x['Troponin Elevation index'][atti_ls[i+1]:] = 'up'
        elif after < before*0.8:
            x['Troponin Elevation index'][atti_ls[i+1]:] = 'down'
        elif (after >= before*0.8) & (after <= before*1.2) :
            x['Troponin Elevation index'][atti_ls[i+1]:] = 'maintain'          
    x['Troponin Elevation index'].fillna('Unk', inplace=True)

    #creatinine
    actual_crea_test_index = x[x['Actual Creatinine test']==1].index
    acti_ls = actual_crea_test_index.tolist()
    #index를 list로 변환해서 길이만큼
    if len(acti_ls)!=0:
        x['Creatinine Elevation index'][acti_ls[0]:] = 'maintain'
    for i in range(len(acti_ls)-1): 
        before = x['Creatinine'][acti_ls[i]]
        after = x['Creatinine'][acti_ls[i+1]]
        if after > before*1.1:
            x['Creatinine Elevation index'][acti_ls[i+1]:] = 'up'
        elif after < before*0.9:
            x['Creatinine Elevation index'][acti_ls[i+1]:] = 'down'
        elif (after >= before*0.9) & (after <= before*1.1) :
            x['Creatinine Elevation index'][acti_ls[i+1]:] = 'maintain'
    x['Creatinine Elevation index'].fillna('Unk', inplace=True)

    #lactate
    actual_Lactate_test_index = x[x['Actual Lactate test']==1].index
    alti_ls = actual_Lactate_test_index.tolist()
    #index를 list로 변환해서 길이만큼
    if len(alti_ls)!=0:
        x['Lactate Elevation index'][alti_ls[0]:] = 'maintain'
    for i in range(len(alti_ls)-1):
        before = x['Lactate'][alti_ls[i]]
        after = x['Lactate'][alti_ls[i+1]]
        if after > before*1.2:
            x['Lactate Elevation index'][alti_ls[i+1]:] = 'up'
        elif after < before*0.8:
            x['Lactate Elevation index'][alti_ls[i+1]:] = 'down'
        elif (after >= before*0.8) & (after <= before*1.2) :
            x['Lactate Elevation index'][alti_ls[i+1]:] = 'maintain'
    x['Lactate Elevation index'].fillna('Unk', inplace=True)
        
    return x

result_1hr_elevation = result_1hr.groupby('stay_id').apply(add_elevation_index)
result_1hr_elevation.reset_index(drop=True, inplace=True)
print(result_1hr_elevation['stay_id'].unique().shape)
print(result_1hr_elevation.shape)


In [ ]:
result_1hr_elevation.to_csv('./1hr shock Dataset elevation labeled.csv', index=False)

In [ ]:
import pandas as pd
result_1hr_elevation = pd.read_csv('./1hr shock Dataset elevation labeled.csv')

In [ ]:
result_1hr_elevation.isnull().sum()

In [ ]:
def find_readmission(x):
    x.reset_index(drop=True, inplace=True)
    stay_id_num = len(x['stay_id'].unique())

    if stay_id_num < 2:
        x['Readmission'] = 0

    else:
        x['Readmission'] = 1
        id_ls = x['stay_id'].unique().tolist()
        new_df = pd.DataFrame(columns=['stay_id', 'time'])

        for stay_id in id_ls:
            first_adm_time = x.loc[x['stay_id'] == stay_id, 'time'].min()
            new_df = new_df.append({'stay_id': stay_id, 'time': first_adm_time}, ignore_index=True)

        new_df = new_df.sort_values(by=['time'], ascending=True)
        first_adm_index = x[x['stay_id'] == new_df.iloc[0]['stay_id']].index
        x.loc[first_adm_index, 'Readmission'] = 0

    return x


result_1hr_elevation = result_1hr_elevation.groupby('hadm_id').apply(find_readmission)
result_1hr_elevation = result_1hr_elevation.sort_values(by=['stay_id','time'], ascending=[True, True])
result_1hr_elevation.reset_index(drop=True, inplace=True)

In [ ]:
print(result_1hr_elevation['hadm_id'].unique().shape)
print(result_1hr_elevation['stay_id'].unique().shape)

In [ ]:
result_1hr_elevation[result_1hr_elevation['Readmission']==1].stay_id.unique().shape

Ambiguous data

In [ ]:
def Ambiguous_data(x): 
    x.reset_index(drop=True, inplace=True)
    column_list = ['Heart rate', 'Blood pressure systolic', 'Blood pressure diastolic', 'Respiratory rate', 'SpO2', 'Temperature', 
                    'Hemoglobin', 'Creatinine', 'Troponin', 'Lactate', 'Shock index']
    
    rows_with_nan = x[x[column_list].isna().any(axis=1)]

    return rows_with_nan


ambiguous_data = result_1hr_elevation.groupby('stay_id').apply(Ambiguous_data)
print(ambiguous_data.shape)
#imputated_data = 0
ambiguous_data.reset_index(drop=True, inplace=True)
print(ambiguous_data['stay_id'].unique().shape)

ambiguous_data.to_csv('./Data preprocessed 4/1hr ambiguous Dataset.csv', index=False)

In [ ]:
ambiguous_data.isnull().sum()

Non Ambiguous data

In [ ]:
def non_Ambiguous_data(x): 
    x.reset_index(drop=True, inplace=True)
    column_list = ['Heart rate', 'Blood pressure systolic', 'Blood pressure diastolic', 'Respiratory rate', 'SpO2', 'Temperature', 
                    'Hemoglobin', 'Creatinine', 'Troponin', 'Lactate', 'Shock index']
    
    rows_without_nan = x[x[column_list].notna().all(axis=1)]

    return rows_without_nan


non_ambiguous_data = result_1hr_elevation.groupby('stay_id').apply(non_Ambiguous_data)
print(non_ambiguous_data.shape)
#imputated_data = 0
non_ambiguous_data.reset_index(drop=True, inplace=True)
print(non_ambiguous_data['stay_id'].unique().shape)

In [ ]:
non_ambiguous_data.isnull().sum()

In [ ]:
non_ambiguous_data.to_csv('./1hr non_ambiguous_data Dataset.csv', index=False)

In [ ]:
import pandas as pd
import itertools

ambiguous_data = ambiguous_data.replace({'gender' : 'F'}, 0)
ambiguous_data = ambiguous_data.replace({'gender' : 'M'}, 1)

ambiguous_data = ambiguous_data.replace({'Troponin Elevation index' : 'maintain'}, 0)
ambiguous_data = ambiguous_data.replace({'Troponin Elevation index' : 'up'}, 2)
ambiguous_data = ambiguous_data.replace({'Troponin Elevation index' : 'down'}, -2)
ambiguous_data = ambiguous_data.replace({'Troponin Elevation index' : 'Unk'}, 100)

ambiguous_data = ambiguous_data.replace({'Creatinine Elevation index' : 'maintain'}, 0)
ambiguous_data = ambiguous_data.replace({'Creatinine Elevation index' : 'up'}, 2)
ambiguous_data = ambiguous_data.replace({'Creatinine Elevation index' : 'down'}, -2)
ambiguous_data = ambiguous_data.replace({'Creatinine Elevation index' : 'Unk'}, 100)

ambiguous_data = ambiguous_data.replace({'Lactate Elevation index' : 'maintain'}, 0)
ambiguous_data = ambiguous_data.replace({'Lactate Elevation index' : 'up'}, 2)
ambiguous_data = ambiguous_data.replace({'Lactate Elevation index' : 'down'}, -2)
ambiguous_data = ambiguous_data.replace({'Lactate Elevation index' : 'Unk'}, 100)

ambiguous_data['Elevation index type'] = None

In [ ]:
ambiguous_data['Data type'] = 'Ambiguous'

In [ ]:
ambiguous_data.to_csv('./1hr Shock Research ambiguous Dataset_type27.csv', index=False)

In [ ]:
ambiguous_data

In [ ]:
import pandas as pd
import itertools
non_ambiguous_data = pd.read_csv('./Data preprocessed 4/1hr non_ambiguous_data Dataset.csv')

data2 = non_ambiguous_data[['Troponin Elevation index', 'Creatinine Elevation index', 'Lactate Elevation index']]

# 가능한 조합 생성
combinations = list(itertools.product(['up', 'down', 'maintain'], repeat=len(data2.columns)))

# 조합에 따라 값을 할당
data2['Elevation index type'] = [combinations.index(tuple(row)) for row in data2.itertuples(index=False, name=None)]

result_df = pd.DataFrame(data2)
# 결과 확인
combination_df = pd.DataFrame(combinations, columns=['Troponin Elevation index','Creatinine Elevation index','Lactate Elevation index'])
combination_df.reset_index(drop=False, inplace=True)
combination_df.rename(columns={'index':'Elevation index type'}, inplace=True)
combination_df.to_csv('./Elevation index type_27.csv', index=False)

print(result_df['Elevation index type'].unique().shape)

elevation_index_ratio = result_df['Elevation index type'].value_counts()/len(result_df)

elevation_index_ratio_df = pd.DataFrame(elevation_index_ratio)
elevation_index_ratio_df.reset_index(drop=False, inplace=True)
elevation_index_ratio_df.rename(columns={'Elevation index type':'Ratio'}, inplace=True)
elevation_index_ratio_df.rename(columns={'index':'Elevation index type'}, inplace=True)

under1 = elevation_index_ratio_df[elevation_index_ratio_df['Ratio']<0.01]
over1_under10 = elevation_index_ratio_df[(elevation_index_ratio_df['Ratio']>=0.01)&(elevation_index_ratio_df['Ratio']<0.10)]
over10_under20 = elevation_index_ratio_df[(elevation_index_ratio_df['Ratio']>=0.10)&(elevation_index_ratio_df['Ratio']<0.20)]
over20 = elevation_index_ratio_df[elevation_index_ratio_df['Ratio']>=0.20]

print(under1['Elevation index type'].unique().shape)
print(over1_under10['Elevation index type'].unique().shape)
print(over10_under20['Elevation index type'].unique().shape)
print(over20['Elevation index type'].unique().shape)

print(under1['Ratio'].sum())
print(over1_under10['Ratio'].sum())
print(over10_under20['Ratio'].sum())
print(over20['Ratio'].sum())

non_ambiguous_data['Elevation index type'] = result_df['Elevation index type']

In [ ]:
non_ambiguous_data = non_ambiguous_data.replace({'gender' : 'F'}, 0)
non_ambiguous_data = non_ambiguous_data.replace({'gender' : 'M'}, 1)

non_ambiguous_data = non_ambiguous_data.replace({'Troponin Elevation index' : 'maintain'}, 0)
non_ambiguous_data = non_ambiguous_data.replace({'Troponin Elevation index' : 'up'}, 2)
non_ambiguous_data = non_ambiguous_data.replace({'Troponin Elevation index' : 'down'}, -2)

non_ambiguous_data = non_ambiguous_data.replace({'Creatinine Elevation index' : 'maintain'}, 0)
non_ambiguous_data = non_ambiguous_data.replace({'Creatinine Elevation index' : 'up'}, 2)
non_ambiguous_data = non_ambiguous_data.replace({'Creatinine Elevation index' : 'down'}, -2)

non_ambiguous_data = non_ambiguous_data.replace({'Lactate Elevation index' : 'maintain'}, 0)
non_ambiguous_data = non_ambiguous_data.replace({'Lactate Elevation index' : 'up'}, 2)
non_ambiguous_data = non_ambiguous_data.replace({'Lactate Elevation index' : 'down'}, -2)

In [ ]:
non_ambiguous_data['Data type'] = 'Non Ambiguous'
non_ambiguous_data

In [ ]:
non_ambiguous_data.to_csv('./1hr Shock Research non_ambiguous Dataset_type27.csv', index=False)

In [ ]:
'''#1hr
import seaborn as sns
import matplotlib.pyplot as plt
x = ['Bin1','Bin2','Bin3','Bin4']
y = [0.0407, 0.5835, 0, 0.3757]
z = [10, 16, 0, 1]

sns.set_theme(style='whitegrid')
sns.barplot(x=y,y=x, width=0.5)
plt.savefig("1hr Elevation index combination distribution_type27")'''

Concat ambiguous and non ambiguous

In [ ]:
ambiguous_data = pd.read_csv('./1hr Shock Research ambiguous Dataset_type27.csv')
non_ambiguous_data = pd.read_csv('./1hr Shock Research non_ambiguous Dataset_type27.csv')

In [ ]:
output = pd.concat([ambiguous_data, non_ambiguous_data], axis=0, ignore_index=True)
output = output.sort_values(by=['stay_id', 'time'], ascending=[True,True])
output.reset_index(drop=True, inplace=True)
output.to_csv('./1hr Shock Research type27 for Cont_delta.csv', index=False)

In [ ]:
output.columns